<a href="https://colab.research.google.com/github/DhruvaBansal00/ConfidentMT/blob/master/BoostedBinaryClassifers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
!ls
%cd drive/My Drive/ConfidentMachineTranslation/flores
!ls
!pwd

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
drive  sample_data
/content/drive/My Drive/ConfidentMachineTranslation/flores
'=0.5.0'			 Ensembling
 analysis			 FLORES.ipynb
 Analysis			 LanguageAnalysis.ipynb
 backward_models		 language_models
 BoostedBinaryClassifers.ipynb	 LM_Thresholding.ipynb
 checkpoints			 NCD_Analysis.ipynb
 ClassificationDataset		 NNClassification.ipynb
 configs			 noisychannel
 data				 NoisyChannel.ipynb
 data-bin			 Resources
 Ensembles			 scripts
/content/drive/My Drive/Confid

In [2]:
# prints how much GPU RAM is available
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

  Created wheel for gputil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7413 sha256=8d617771c9b0b2aad6c9b84c6f42d015bc9341b147b53cc57d4d5236b2f5f9f9
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
Gen RAM Free: 12.7 GB  | Proc size: 159.7 MB
GPU RAM Free: 11441MB | Used: 0MB | Util   0% | Total 11441MB


In [8]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
import numpy as np
from itertools import zip_longest
from sklearn.svm import SVC

def printDatasetClassProp(Y): 
    classes = {}
    total = len(Y)
    for i in Y:
        if i in classes:
            classes[i] += 1
        else:
            classes[i] = 1
    
    for cls in classes:
        print("Proportion in class " + str(cls) + " = " + str(classes[cls]/total))

def datasetReader(featureFile, labelFile):
    files = [featureFile, labelFile]

    X = []
    Y = []

    for lines in zip_longest(*files, fillvalue=''):
        currX, currY = lines[0], float(lines[1].strip("\n"))
        Xarr = []
        features = currX.split()
        for feature in features:
            Xarr.append(float(feature.strip(",").strip("\n")))
        X.append(Xarr)
        Y.append(currY)
    
    return np.array(X), np.array(Y)


def computeSimilarity(o1, o2):
    total = len(o1)
    same = 0
    for i in range(len(o1)):
        if o1[i] == o2[i]:
            same += 1
    print(same/total)

def trainGaussianProcessClassifier(X, Y):
    print("Training Gaussian Process Classifier")
    length_scale = [1 for i in range(len(X[0]))]
    clf = GaussianProcessClassifier(1.0 * RBF(length_scale), warm_start=True, random_state=42, n_jobs=-1)
    clf.fit(X, Y)
    return clf

def trainEnsembleClassifier(X, Y, maxDepth=8, estimators=100):
    print("Training an ensemble of Random Forest, AdaBoosting and Gradient Boosting Classifiers")

    estimators = [
     ('rf', RandomForestClassifier(max_depth=maxDepth, random_state=42)),
     ('ada', AdaBoostClassifier(n_estimators=estimators, random_state=42)),
     ('grad', GradientBoostingClassifier(random_state=42))]
    clf = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())
    clf.fit(X, Y)
    return clf


def trainRandomForestClassifier(X, Y, maxDepth=8):
    print("Training Random Forest classifier")
    clf = RandomForestClassifier(max_depth=maxDepth, random_state=42)
    clf.fit(X, Y)
    return clf

def trainAdaBoostClassifier(X, Y, estimators=100):
    print("Training AdaBoosted Decision Tree classifier")
    clf = AdaBoostClassifier(n_estimators=estimators, random_state=42)
    clf.fit(X, Y)
    return clf

def trainGradientBoostingClassifier(X, Y):
    print("Training Graident Boosted classifier")
    clf = GradientBoostingClassifier(random_state=42)
    clf.fit(X, Y)
    return clf

def trainSVM(X, Y):
    print("Training SVM classifier")
    clf = SVC(gamma='auto')
    clf.fit(X, Y)
    return clf

def calculateAccuracy(predictedClasses, groundTruth):
    correct_accepted = 0
    total_accepted = 0

    correct_rejected = 0
    total_rejected = 0

    for i in range(len(predictedClasses)):
        if groundTruth[i] == 1:
            total_accepted += 1
            if predictedClasses[i] == groundTruth[i]:
                correct_accepted += 1
        else:
            total_rejected += 1
            if predictedClasses[i] == groundTruth[i]:
                correct_rejected += 1


    print("Correctly accepted = " + str(correct_accepted/total_accepted))
    print("Incorrectly accepted = " + str(1 - correct_rejected/total_rejected))
    print("Correctly rejected = " + str(correct_rejected/total_rejected))
    print("Incorrectly rejected = " + str(1 - correct_accepted/total_accepted))

    print("Total Accuracy = " + str((correct_accepted + correct_rejected)/(total_accepted + total_rejected)))


trainset = "valid"
testset = "test"
bleuThresholdTrain = 10
bleuThresholdTest = 10

trainFeatures = open("ClassificationDataset/"+str(bleuThresholdTrain)+"BLEU/"+trainset+"/features.txt")
testFeatures = open("ClassificationDataset/"+str(bleuThresholdTest)+"BLEU/"+testset+"/features.txt")

trainLabels = open("ClassificationDataset/"+str(bleuThresholdTrain)+"BLEU/"+trainset+"/result.txt")
testLabels = open("ClassificationDataset/"+str(bleuThresholdTest)+"BLEU/"+testset+"/result.txt")

trainX, trainY = datasetReader(trainFeatures, trainLabels)
testX, testY = datasetReader(testFeatures, testLabels)

printDatasetClassProp(trainY)


classifiers = [trainRandomForestClassifier, trainAdaBoostClassifier, trainGradientBoostingClassifier, trainSVM, trainGaussianProcessClassifier]
outputs = []

for classifier in classifiers:
    print("#################################################")
    curr = classifier(trainX, trainY)
    predictions = np.array(curr.predict(testX))
    calculateAccuracy(predictions, testY)
    outputs.append(predictions)
    print("#################################################")

for output_1 in outputs:
    for output_2 in outputs:
        computeSimilarity(output_1, output_2)


Proportion in class 1.0 = 0.5287221570926143
Proportion in class 0.0 = 0.4712778429073857
#################################################
Training Random Forest classifier
Correctly accepted = 0.7030965391621129
Incorrectly accepted = 0.2861952861952862
Correctly rejected = 0.7138047138047138
Incorrectly rejected = 0.2969034608378871
Total Accuracy = 0.7075837742504409
#################################################
#################################################
Training AdaBoosted Decision Tree classifier
Correctly accepted = 0.6964177292046144
Incorrectly accepted = 0.32407407407407407
Correctly rejected = 0.6759259259259259
Incorrectly rejected = 0.3035822707953856
Total Accuracy = 0.6878306878306878
#################################################
#################################################
Training Graident Boosted classifier
Correctly accepted = 0.7061323618700668
Incorrectly accepted = 0.29461279461279466
Correctly rejected = 0.7053872053872053
Incorrectly rejected

In [4]:
set1 = "train"
set2 = "valid"


set1Features = open("ClassificationDataset/"+set1+"/features.txt")
set2Features = open("ClassificationDataset/"+set2+"/features.txt")

set1Labels = open("ClassificationDataset/"+set1+"/result.txt")
set2Labels = open("ClassificationDataset/"+set2+"/result.txt")


_,set1Labels = datasetReader(set1Features, set1Labels)
_,set2Labels = datasetReader(set2Features, set2Labels)


num_reject = 0
for label in set1Labels:
    if label == 0:
        num_reject += 1

num_accept = 0
for label in set2Labels:
    if label == 1:
        num_accept += 1

print("Num rejected in set 1 = " + str(num_reject))
print("Num accept in set 2 = " + str(num_accept))

FileNotFoundError: ignored